In [21]:
import pandas as pd
from viruscope import readfa
import os
import os.path as op
import glob
from collections import Counter

In [11]:
wd = "/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/"

In [12]:
vogs = glob.glob(op.join(wd, "indiv_vogs", "*.faa"))

In [13]:
len(vogs)

22318

In [14]:
test = vogs[0]
test

'/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/indiv_vogs/VOG00001.faa'

In [16]:
rdf = pd.read_csv(op.join(wd, 'vog.annotations.tsv'), sep="\t")

In [23]:
Counter(rdf['ConsensusFunctionalDescription']).most_common()

[('REFSEQ hypothetical protein', 16313),
 ('Genome polyprotein', 69),
 ('REFSEQ structural protein', 54),
 ('REFSEQ hypothetical protein 1', 31),
 ('Capsid protein', 27),
 ('REFSEQ hyphothetical protein', 22),
 ('REFSEQ putative tail fiber protein', 20),
 ('REFSEQ F-box domain', 20),
 ('REFSEQ holin', 18),
 ('REFSEQ virion structural protein', 18),
 ('REFSEQ putative capsid protein', 17),
 ('Major capsid protein', 17),
 ('REFSEQ putative virion structural protein', 17),
 ('REFSEQ minor tail protein', 15),
 ('REFSEQ RNA-dependent RNA polymerase', 15),
 ('REFSEQ tail assembly chaperone', 15),
 ('Nucleoprotein', 15),
 ('Replicase polyprotein 1a', 14),
 ('Phosphoprotein', 14),
 ('Matrix protein', 13),
 ('REFSEQ HNH endonuclease', 13),
 ('REFSEQ gp25', 12),
 ('REFSEQ tail fiber protein', 12),
 ('REFSEQ head-to-tail connector protein', 12),
 ('Movement protein', 12),
 ('REFSEQ hypothetical protein 2', 11),
 ('RNA-directed RNA polymerase', 11),
 ('REFSEQ terminase small subunit', 11),
 ('REFS

In [24]:
with open(op.join(wd, "vog_proteins_nr.fasta"), "w") as oh:
    for v in vogs:
        seqs = []

        vog = op.basename(v).split(".")[0]
        fcat = rdf[rdf['#GroupName'] == vog]['FunctionalCategory'].values[0]
        fdesc = rdf[rdf['#GroupName'] == vog]['ConsensusFunctionalDescription'].values[0]

        for name, seq in readfa(open(v)):
            if seq not in seqs:
                print(">{name}__{fcat}__{vog}:{fdesc}".format(name=name.replace(" ","_"), vog=vog, fcat=fcat, fdesc=fdesc.replace(" ","_")), file=oh)
                for i in range(0, len(seq), 60):
                    print(seq[i:i+60], file=oh)

                seqs.append(seq)

In [25]:
!head {op.join(wd, "vog_proteins_nr.fasta")} -n200

>10243.NP_619808.1_CPXV019_protein_[Cowpox_virus]__Xu__VOG00001:Putative_ankyrin_repeat_protein_FPV162
MNSLIENSVLHVRKLLYTIHFNDIDHAPTTATSRDCENQYDSYDIYVRSVFKNNLVLERK
YYDIVYKLINSTISHTINYDDQTYDNNIKSLLEVTTRFHRHYITSLSRKKKSSIDSIEQM
YSQIIKDITTELINDENVVIDVDDLCLIDYDDNKDDNSDSDSDSDDEDEDSRYENSGKYS
SNIFSVYFKAHNTIGIDIKLVKWMIAKGVDINYKNEYGSGILHTYLGNMYVDTDVLELLC
RSGKNINEQNNHLITPLHSYLRRDESISASVLKKVIELGADRNLRCCMKLTPIMTYIIYN
AMNIDEEIIKIYMDDNKITNIPEILYLYIQMAQYVDIDVVKHFLKNGVILDYKDSVGRTC
LHNYMLRTYINIDIVKLLLEHGNYLNEVDDFGNTVLHTYLSRLCIVNKNLTYDSSINNNI
NIEVVKYLIRSGSDISLVNNLGYTPLTSYICTAQNYMYHDIIDCLMSDKVLNMVKHRIIQ
DLITRKDISLRIIHHVITKYNIPVDSYTDEYKKCNYKTNDAYHFRMIKYYNNNILPFSGM
TAFHVAVCIRNDVIMKYLLSIGYSIDKPTIKGLTALMLIFRRDRYTNCHIVKMLLDNRPS
IDNIIIFLDKCYKNESLFSLCSTKNTNLYGMMALAFMLIGVNNCNKFIERIKHDLETQNT
TNYRSLSHTLDVLVDIRDNISLLMSTHINNYSTISIYDILISRTYNIDITSCIEICKMYT
SDKYAFYDIINGYIDNAIKTNSIVKDIIYYLRTYPDMCIPTSLLRNCIIDMYDLSGFRET
LKRNTSAILPTLNHLN
>10243.NP_619979.1_CPXV198_protein_[Cowpox_virus]__Xu__VOG00001:Putative_ankyrin_repeat

In [26]:
!grep -c ">" {op.join(wd, "vog_proteins_nr.fasta")}

217028


In [27]:
!makeblastdb -in {op.join(wd, "vog_proteins_nr.fasta")} -dbtype prot 



Building a new DB, current time: 04/13/2018 12:18:47
New DB name:   /mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog_proteins_nr.fasta
New DB title:  /mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog_proteins_nr.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog_proteins_nr.fasta
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 217028 sequences in 11.0532 seconds.


In [6]:
!makeblastdb --help

USAGE
  makeblastdb [-h] [-help] [-in input_file] [-input_type type]
    -dbtype molecule_type [-title database_title] [-parse_seqids]
    [-hash_index] [-mask_data mask_data_files] [-mask_id mask_algo_ids]
    [-mask_desc mask_algo_descriptions] [-gi_mask]
    [-gi_mask_name gi_based_mask_names] [-out database_name]
    [-max_file_sz number_of_bytes] [-logfile File_Name] [-taxid TaxID]
    [-taxid_map TaxIDMapFile] [-version]

DESCRIPTION
   Application to create BLAST databases, version 2.2.31+

Use '-help' to print detailed descriptions of command line arguments

Error: Unknown argument: "-help"
